In [1]:
import requests
import os
import sys
import tarfile
import pickle

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import random
import numpy as np
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

from chainer.datasets import split_dataset_random, TupleDataset

## Load dataset

Let's load CIFAR-10 dataset.

### Download tar.gz file

In [3]:
def maybe_download(url, save_dir = './datasets/'):    
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    filename = url.split('/')[-1]
    file_path = os.path.join(save_dir, filename)
    if os.path.exists(file_path):
        return file_path
    
    r = requests.get(url)
    if r.status_code == 200:    
        with open(file_path, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
    else:
        r.raise_for_status()
        
    return file_path

In [4]:
url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
file_path = maybe_download(url)

### Extract tar file

In [5]:
def extract_tar(file_path):
    target_dir = os.path.dirname(file_path)

    with tarfile.open(file_path) as tar:
        for n in tar.getnames():
            member = os.path.abspath(os.path.join(target_dir, n))
            print(member)
            
            if not member.startswith(os.path.abspath(target_dir)):
                sys.exit(1)

        tar.extractall(path=target_dir)

In [6]:
extract_tar(file_path)

/tmp/notebooks/chainer/datasets/cifar-10-batches-py
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/data_batch_4
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/readme.html
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/test_batch
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/data_batch_3
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/batches.meta
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/data_batch_2
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/data_batch_5
/tmp/notebooks/chainer/datasets/cifar-10-batches-py/data_batch_1


In [7]:
def unpickle(file):

    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

### Unpickel files

In [11]:
pickled_data_dir = "/tmp/notebooks/chainer/datasets/cifar-10-batches-py"

def load_cifar10(data_dir):
    ## Train data
    train_data = np.empty((0, 3 * 32 * 32), dtype=np.float32)
    train_labels = np.empty((0, 1), dtype=np.int32)

    for i in range(1,6):
        file = os.path.join(pickled_data_dir, "data_batch_{}".format(i))
        batch = unpickle(file)

        train_data = np.vstack((train_data, batch[b'data']))
        train_labels = np.vstack((train_labels, np.array(batch[b'labels']).reshape(-1, 1)))

    ## Test data
    file = os.path.join(pickled_data_dir, "test_batch")
    batch = unpickle(file)

    test_data = np.empty((0, 3 * 32 * 32), dtype=np.float32)
    test_labels = np.empty((0, 1), dtype=np.int32)
    test_data = np.vstack((test_data, batch[b'data']))
    test_labels = np.vstack((test_labels, np.array(batch[b'labels']).reshape(-1, 1)))
    
    ## Meta
    file = os.path.join(pickled_data_dir, "batches.meta")
    meta = unpickle(file)
    label_names = meta[b'label_names']
    
    train_labels = train_labels.astype(np.int32)
    test_labels = test_labels.astype(np.int32)
    
    return train_data, train_labels, test_data, test_labels, label_names

In [12]:
train_data, train_labels, test_data, test_labels, label_names = load_cifar10(pickled_data_dir)

In [13]:
print("train_data shape: {}".format(train_data.shape))
print("train_labels shape: {}".format(train_labels.shape))
print("test_data shape: {}".format(test_data.shape))
print("test_labels shape: {}".format(test_labels.shape))
print(label_names)

print("\n------------------\n")
print("train_data type: {}".format(train_data.dtype))
print("train_labels type: {}".format(train_labels.dtype))
print("test_data type: {}".format(test_data.dtype))
print("test_labels type: {}".format(test_labels.dtype))

train_data shape: (50000, 3072)
train_labels shape: (50000, 1)
test_data shape: (10000, 3072)
test_labels shape: (10000, 1)
[b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']

------------------

train_data type: float32
train_labels type: int32
test_data type: float32
test_labels type: int32


### See some examples

In [12]:
def show_image(vector):
    image = np.moveaxis(vector.reshape(3, 32, 32), 0, -1)
    plt.imshow(image.astype(int))

In [13]:
show_image(train_data_orig[0])

NameError: name 'train_data_orig' is not defined

## Prepare iterators

In [14]:
train = TupleDataset(train_data, train_labels)
valid = TupleDataset(test_data, test_labels)

batch_size = 128

train_iter = iterators.SerialIterator(train, batch_size)
valid_iter = iterators.SerialIterator(valid, batch_size, repeat=False, shuffle=False)

## Build the model

In [23]:
class LeNet5LikeConvNet(Chain):
    def __init__(self):
        super(LeNet5LikeConvNet, self).__init__()
        
        with self.init_scope():
            self.c1 = L.Convolution2D(3, 6, ksize=5, stride=1)
            self.c3 = L.Convolution2D(6, 16, ksize=5, stride=1)
            self.c5 = L.Convolution2D(16, 120, ksize=5, stride=1)
            self.f6  = L.Linear(None, 84)
            self.out = L.Linear(84, 10)
    
    def normalize(self, x):
        x = F.reshape(x, (-1, 3, 32, 32))
        x = F.cast(X, np.float32)        
    
    def __call__(self, x):
        X = F.reshape(x, (-1, 3, 32, 32))
        X = F.cast(X, np.float32)
        
        self.c1(X)

        a1 = F.tanh(self.c1(X))                                # shape: (m, 6, 28, 28)
        a2 = F.max_pooling_2d(a1, 2, stride=2)  # shape: (m, 6, 14, 14)
        a3 = F.tanh(self.c3(a2))                              # shape: (m, 16, 10, 10)
        a4 = F.max_pooling_2d(a3, 2, stride=2)  # shape: (m, 16, 5, 5)
        a5 = F.tanh(self.c5(a4))                              # shape: (m, 120, 1, 1)
        a6 = F.tanh(self.f6(a5))                              # shape: (m, 84)
        out =self.out(a6)                                         # shape: (m, 10) 
        
        if chainer.config.train:
            return out

        return F.softmax(out)

In [24]:
x0, y0 = train[0]
model = LeNet5LikeConvNet()

print(y0[0])
print(type(y0[0]))
print(x0)
print(type(x0[1025]))

print(type(model(x0)))

# print(model(x0).shape)
# print(model(x0))

6.0
<class 'numpy.float64'>
[ 59.  43.  50. ... 140.  84.  72.]
<class 'numpy.float64'>
<class 'chainer.variable.Variable'>
